In [1]:
!pip install jiwer pyctcdecode "pypi-kenlm" --upgrade -qq

In [2]:
# for cpu inference
# !pip uninstall onnxruntime-gpu -y
# !pip install onnxruntime -qq

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('..')

In [3]:
import onnxruntime as rt
from sk import predict
from pathlib import Path
import jiwer
import pandas as pd
import string
from fastcore.basics import partialler
from IPython.display import Audio
import multiprocessing
pd.set_option('max_colwidth', 400)

In [4]:
if 0:
    sys.path.append('../../neem')
    from model import get_model
    import torch
    model = get_model("conformer_small",cuda=False)
    model.load_state_dict(torch.load("/content/MyDrive/neem/models/Conformer_3_5.055281162261963.pth"))
    model = model.eval();
elif 1:
    import tensorflow as tf
    tf_model = tf.saved_model.load('/content/model')
    model = tf_model.signatures["serving_default"]
else:
    sess_options = rt.SessionOptions()
    # Set graph optimization level to ORT_ENABLE_EXTENDED to enable bert optimization.
    sess_options.graph_optimization_level = rt.GraphOptimizationLevel.ORT_ENABLE_EXTENDED
    # Use OpenMP optimizations. Only useful for CPU, has little impact for GPUs.
    sess_options.intra_op_num_threads = multiprocessing.cpu_count()
    model = rt.InferenceSession("/content/output.onnx",sess_options)

In [5]:
from pyctcdecode import build_ctcdecoder
import kenlm
kenlm_model = kenlm.Model("/content/out.trie.klm")
labels = list(
            string.ascii_lowercase  # + string.digits
        ) + [" ","_"]
decoder = build_ctcdecoder(
    labels,
    kenlm_model, 
    alpha=0.5,
    beta=1.0, 
    ctc_token_idx=labels.index("_")
)

In [6]:
decoder.decode_beams = partialler(decoder.decode_beams,hotwords=None,hotword_weight=10.) # reset hotwords

In [7]:
fn = "/content/test-bahasa/"
preds,files,entropy,timesteps = predict(model,fn,decoder)

100%|██████████| 780/780 [08:52<00:00,  1.46it/s]


In [8]:
data = []
for i,j in zip(preds,files):
    label = j.with_suffix('.txt').read_text()
    char_label = [char for char in label]
    char_i = [char for char in i]
    data.append([j,label,i,jiwer.compute_measures(label,i)['wer'],jiwer.compute_measures(char_label,char_i)['wer']])

In [9]:
df = pd.DataFrame(data)
df.columns = ["path","label","pred","wer","cer"]
df = df.sort_values("cer",ascending=False)
df.head(50)

,path,label,pred,wer,cer
276,/content/test-bahasa/wattpad-audio-wattpad-646.wav,who are you,ayu,1.000000,0.666667
428,/content/test-bahasa/-home-husein-speech-bahasa-streaming-iaitu paus odontoceti paus.wav,iaitu paus odontoceti paus,iaitu power ordonterceti powers,0.750000,0.478261
104,/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-biperforate.wav,tolong sebut biperforate,dolok sebut bipufarad,0.666667,0.363636
338,/content/test-bahasa/-home-husein-speech-bahasa-haqkiem-LJ118-000005.wav,boleh fuckers quora sila tutup fuck tentang iq,boleh pakar sekolah sila tutup pak tentang aku,0.500000,0.358974
243,/content/test-bahasa/wattpad-audio-wattpad-638.wav,itu andra batin libra,itu anda makin liberal,0.750000,0.277778
563,/content/test-bahasa/-home-husein-speech-bahasa-sebut-perkataan-man-ampe.wav,sebut perkataan ampe,sebuk pakatan ampa,1.000000,0.277778
180,/content/test-bahasa/-home-husein-speech-bahasa-streaming-pasang surut planet utama.wav,pasang surut planet utama,pasal surut klinik utama,0.500000,0.272727
383,/content/test-bahasa/wattpad-audio-wattpad-88.wav,udah ga cape kita caw yuk,udagacapi kita chayok,0.833333,0.250000
14,/content/test-bahasa/-home-husein-speech-bahasa-tolong-sebut-mesaticephalous.wav,tolong sebut mesaticephalous,tolong sebut mesatusapulus,0.333333,0.230769
754,/content/test-bahasa/iium-audio-iium-6.wav,petang tu mak bawa aku jumpa uncle man,petang tu mak bawa ke jumpa akaman,0.375000,0.225806


In [10]:
df["wer"].mean(),df["cer"].mean()

(0.13930081308861203, 0.03735004862207611)

In [10]:
df["wer"].mean(),df["cer"].mean()

(0.13930081308861203, 0.03735004862207611)

In [10]:
df["wer"].mean(),df["cer"].mean()

(0.13808961378922666, 0.03655201265588635)

# Grid search